In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv("employee_data.csv")

print(df)

      id groups  age  healthy_eating  active_lifestyle  salary
0      0      A   36               5                 5    2297
1      1      A   55               3                 5    1134
2      2      A   61               8                 1    4969
3      3      O   29               3                 6     902
4      4      O   34               6                 2    3574
..   ...    ...  ...             ...               ...     ...
995  995      O   33               7                 7    2996
996  996      O   21               1                 2     667
997  997      O   49               9                 7    4158
998  998     AB   56               6                 7    2414
999  999      B   64               4                 9     788

[1000 rows x 6 columns]


In [2]:
df1=df[0:200].copy()
del df1['id']
del df1['groups']
#including salary too: Reason : Kmeans predictions requires all the features and its is needed for better clustering.
#clustering has been done using all the relevant features inlcuding salary for better clustering.


In [3]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaled_df1 = scaler.fit_transform(df1)
scaled_df1 = pd.DataFrame(scaled_df1, columns=['age', 'healthy_eating', 'active_lifestyle','salary'])
scaled_df1

,age,healthy_eating,active_lifestyle,salary
0,-0.447355,-0.013078,-0.357260,0.048003
1,0.993280,-1.059299,-0.357260,-1.020373
2,1.448217,1.556254,-2.342035,2.502604
3,-0.978115,-1.059299,0.138934,-1.233497
4,-0.599001,0.510033,-1.845841,1.221104
...,...,...,...,...
195,1.069103,-0.013078,0.635128,-0.378245
196,-1.812167,-1.059299,-0.853454,-0.807249
197,0.689988,1.556254,0.138934,1.436984
198,-0.599001,-1.059299,0.138934,-1.233497


In [4]:
from sklearn.cluster import KMeans
di=0.0
di1=0.0

for i in range(2,21):
    kmeans = KMeans(n_clusters=i,random_state=1)
    kmeans.fit(scaled_df1[0:180])
    labels=kmeans.labels_  
    labels=list(labels)
    centroids=kmeans.cluster_centers_
    #print(centroids)
    min_=centroids[1]-centroids[0]
    min_squared=min_[0]**2+min_[1]**2+min_[2]**2
    min_squared=min_squared**.5
    #dunn's Index
    for j in range(len(centroids)):
        #calculating Minimum distance between clusters
        for k in range(j+1,len(centroids)):
            dist=centroids[k]-centroids[j]
            dist_squared=dist[0]**2+dist[1]**2+dist[2]**2
            dist_squared=dist_squared**.5
            if(dist_squared<min_squared):
                min_squared=dist_squared
    print(i)
    sum_=[]
    for j in range(len(centroids)):
        sum_.append(0)
    for j in range(len(labels)):
        #Calculating Delta
        for k in range(j+1,len(labels)):
            if(labels[j]==labels[k]):
                dist1=(scaled_df1.iloc[j]['age']-scaled_df1.iloc[k]['age'])**2+(scaled_df1.iloc[j]['healthy_eating']-scaled_df1.iloc[k]['healthy_eating'])**2+(scaled_df1.iloc[j]['active_lifestyle']-scaled_df1.iloc[k]['active_lifestyle'])**2+(scaled_df1.iloc[j]['salary']-scaled_df1.iloc[k]['salary'])**2
            
                dist1_squared=dist1**.5
               
                sum_[labels[j]]=sum_[labels[j]]+dist1_squared
                
    for j in range(len(centroids)):
        cn=labels.count(j)
        if(cn==1):
            sum_[j]=0;
        else:
            sum_[j]=(sum_[j])/(cn) # cannot allow (cn-1) as division by zero may occur
            
    
    di=min_squared/max(sum_)
    print(di)

    if (di>di1):
        di1=di
        final_n=i
print("Optimal Number of Clusters = ",final_n)
   
kmeans = KMeans(n_clusters=final_n,random_state=1)
kmeans.fit(scaled_df1[0:180])
cluster_labels_train=kmeans.labels_



2
0.015052324845163648
3
0.026927292951708544
4
0.03000754023261809
5
0.04838479256480625
6
0.059683074994509394
7
0.06640083363309812
8
0.05613955706273812
9
0.08332432697879932
10
0.06881701707140879
11
0.07727913247482819
12
0.07155503765202624
13
0.0766140886299399
14
0.10509861321355689
15
0.09647835588766336
16
0.11459974905392083
17
0.10341622045244582
18
0.0855997693161557
19
0.1375497678809205
20
0.10612981397210548
Optimal Number of Clusters =  19


In [5]:
#adding the cluster no. to train dataframe
train_df=df[0:180].copy()
train_df['Labels']=cluster_labels_train
train_df

,id,groups,age,healthy_eating,active_lifestyle,salary,Labels
0,0,A,36,5,5,2297,3
1,1,A,55,3,5,1134,2
2,2,A,61,8,1,4969,13
3,3,O,29,3,6,902,4
4,4,O,34,6,2,3574,1
...,...,...,...,...,...,...,...
175,175,B,39,3,6,902,14
176,176,A,21,6,6,2646,7
177,177,A,31,2,9,1475,0
178,178,A,44,3,6,902,14


In [6]:
#Setting the cluster no. for test data based on Centroid Distance from test data
centroids=kmeans.cluster_centers_

print(centroids)                               
        

[[-0.17163542 -1.05929878  1.44708186 -1.08116984]
 [-1.18392009  0.3605726  -1.84584132  1.06861039]
 [ 1.04577247 -1.38121294 -0.28092209 -1.08828126]
 [ 0.21338724  0.21111246 -0.14460508  0.18540545]
 [-1.29519262 -0.96418778  0.22915137 -1.00124833]
 [ 0.91204091  0.84631807 -0.42814445  0.95568352]
 [ 0.58889102  0.57978081  1.39595885 -0.0998973 ]
 [-1.3151057   0.33566258 -0.2469943   0.3564617 ]
 [-1.11712374  1.25110597  0.18028379  1.10788217]
 [-0.05740875 -0.98456871 -1.42053226 -0.48730012]
 [ 1.23591292 -1.11160983  0.93284454 -1.16193484]
 [-1.01061065  0.28584253  0.98955242 -0.22548826]
 [ 1.28393407  0.37925512  0.71782718 -0.01346877]
 [ 1.44821698  1.55625376 -2.34203523  2.5026045 ]
 [-0.32328118 -0.82152128  0.00360868 -0.93235047]
 [ 0.60333348  1.92990413 -0.00283539  1.87911168]
 [ 0.3108738   0.04504563 -1.34964742  0.53345265]
 [-1.53920443 -1.16392088 -1.15116986 -0.78611998]
 [-0.15922804  1.24238746 -2.14355767  2.09675018]]


In [7]:
scaled_test_df=scaled_df1[180:200].copy()
cluster_labels_test=[]
features=[]
#getting the relevant features from the test data and storing them in a list for comparison with centroids.
for i in range(20):
    tmp=[]
    for j in range(0,4):
        tmp.append(float(scaled_test_df.iloc[i,j]))
    features.append(tmp)
features=np.array(features)
print(features)

[[-0.52317786 -1.05929878  0.13893429 -1.23349675]
 [-0.90229225  1.03314325  1.1313221   0.47700744]
 [ 0.53834244  1.55625376 -0.35725961  1.65010832]
 [ 1.44821698 -0.53618827  0.6351282  -0.9119734 ]
 [-0.82646937 -0.01307776  0.13893429 -0.16512061]
 [-0.29570923  1.55625376 -0.35725961  1.65010832]
 [-1.50887528 -0.01307776 -0.35725961  0.04800344]
 [-1.4330524  -1.05929878  1.1313221  -1.45396989]
 [ 0.46251956  1.55625376  0.13893429  1.43698428]
 [-0.59900074 -0.53618827  0.13893429 -0.69884936]
 [-0.67482362  0.51003275  1.1313221  -0.05763995]
 [ 0.08340517  1.03314325 -0.35725961  1.11637957]
 [ 0.08340517 -1.05929878  0.13893429 -1.23349675]
 [-0.44735498 -0.53618827 -0.35725961 -0.48572531]
 [-0.59900074 -0.53618827 -0.35725961 -0.48572531]
 [ 1.06910259 -0.01307776  0.6351282  -0.37824465]
 [-1.81216679 -1.05929878 -0.85345352 -0.80724866]
 [ 0.68998819  1.55625376  0.13893429  1.43698428]
 [-0.59900074 -1.05929878  0.13893429 -1.23349675]
 [ 0.61416532 -0.53618827 -1.34

In [8]:
#deciding the cluster no. for test data
test_df=df[180:200].copy()
labels=[]
for i in range(20):
    min_diff_list=centroids[0]-features[i]
    min_diff_squared=(min_diff_list[0]**2+min_diff_list[1]**2+min_diff_list[2]**2)**.5
    index=0;
    for j in range(1,len(centroids)):
        diff_list=centroids[j]-features[i]
        diff_squared=(diff_list[0]**2+diff_list[1]**2+diff_list[2]**2)**.5
        if(diff_squared<min_diff_squared):
            min_diff_sqaured=diff_squared
            index=j
    labels.append(index)
            
test_df['Labels']=labels
test_df

,id,groups,age,healthy_eating,active_lifestyle,salary,Labels
180,180,O,35,3,6,902,14
181,181,O,30,7,8,2764,15
182,182,AB,49,8,5,4041,18
183,183,B,61,4,7,1252,12
184,184,A,31,5,6,2065,14
185,185,A,38,8,5,4041,18
186,186,A,22,5,5,2297,17
187,187,O,23,3,8,662,4
188,188,O,48,8,6,3809,18
189,189,O,34,4,6,1484,14


In [19]:
min_label = int(train_df['Labels'].min())
max_label = int(train_df['Labels'].max())

print(min_label)
print(max_label)

cust_seg = pd.DataFrame(columns=['label', 'min_sal', 'max_sal','min_age','max_age'])

for i in range(min_label,max_label+1):
    rslt_df = train_df[train_df['Labels'] == i]
    max_sal = rslt_df['salary'].max()
    min_sal = rslt_df['salary'].min()
    max_age = rslt_df['age'].max()
    min_age = rslt_df['age'].min()
    cust_seg.loc[i]= [i]+[min_sal]+[max_sal]+[min_age]+[max_age];
    
print(cust_seg)

0
18
   label min_sal max_sal min_age max_age
0      0     556    2406      31      53
1      1    2761    4155      19      34
2      2     667    1716      46      62
3      3    2065    2878      36      58
4      4     779    1484      18      32
5      5    2878    3923      45      64
6      6    1601    3113      43      64
7      7    2065    3110      19      32
8      8    2996    4390      20      34
9      9    1366    1948      30      60
10    10     662    2056      50      64
11    11    1601    2414      20      34
12    12    1833    2646      50      63
13    13    4969    4969      61      61
14    14     670    1716      32      45
15    15    3809    4622      37      62
16    16    2411    3342      37      63
17    17     785    2179      18      27
18    18    3806    4969      31      54


In [20]:
cust_seg.to_excel(r'D:\Notebooks\Salary_Prediction\Salary_prediction\Rules_Result.xlsx',index=None,header=True)